<a href="https://colab.research.google.com/github/medinaalonso/NLP/blob/master/IMDB_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Importing data from KAGGLE*

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"carleider32","key":"985ac65a3d590a561ce195702ae31378"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
allen-institute-for-ai/CORD-19-research-challenge           COVID-19 Open Research Dataset Challenge (CORD-19)    3GB  2020-06-23 00:53:41            994  
roche-data-science-coalition/uncover                        UNCOVER COVID-19 Challenge                          179MB  2020-05-21 18:57:53           4279  
madeleineferguson/farmers-markets-in-the-united-states      Farmers Markets in the United States                816KB  2020-06-10 16:41:46            240  
promptcloud/jobs-on-naukricom                               Jobs On Naukri.com                                    3MB  2020-06-15 10:51:07            245  
josehenriqueroveda/fertilizers-by-product-fao               Fert

In [ ]:
! kaggle datasets download -d mwallerphunware/imbd-movie-reviews-for-binary-sentiment-analysis

 39% 5.00M/12.9M [00:00<00:01, 5.55MB/s]
100% 12.9M/12.9M [00:01<00:00, 13.3MB/s]


In [ ]:
! unzip imbd-movie-reviews-for-binary-sentiment-analysis.zip

Archive:  imbd-movie-reviews-for-binary-sentiment-analysis.zip
  inflating: MovieReviewTrainingDatabase.csv  


# Inicio

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.preprocessing.text import Tokenizer


from tensorflow.compat.v1.nn import rnn_cell
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [14]:
%tensorflow_version 2.x

In [2]:
from csv import reader
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/Redes Neuronales Avanzadas/IMDB Dataset.csv', sep='\t')
print(type(data))

imdb = np.asarray(data)

renglon = str(imdb[3])
length =len(renglon)
clase = renglon[-10:-2]
#print(renglon[2:-12])
#print(clase)
lista = []
for i in range(0,50000):
  renglon = str(imdb[i])
  clase = renglon[-10:-2]
  texto = renglon[2:-12]
  lista.append([clase,texto])
  #print(clase,texto)


#d = pd.DataFrame({'id': data['id'], 'sentiment': p[:,0]})

<class 'pandas.core.frame.DataFrame'>


In [3]:
newlist = np.asarray(lista)
print(newlist.shape)
print(newlist[0][0])

(50000, 2)
positive


In [4]:
for i in range(0,50000):
  clase = newlist[i][0]
  if clase=='positive':
    newlist[i][0] = 0
  if clase=='negative':
    newlist[i][0] = 1

In [5]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
for i in range(0,50000):
  newlist[i][1] = denoise_text(newlist[i][1])
  newlist[i][1] = remove_special_characters(newlist[i][1])

In [6]:
newlist[1]

array(['0',
       'A wonderful little production The filming technique is very unassuming very oldtimeBBC fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece The actors are extremely well chosen Michael Sheen not only has got all the polari but he has all the voices down pat too You can truly see the seamless editing guided by the references to Williams\\ diary entries not only is it well worth the watching but it is a terrificly written and performed piece A masterful production about one of the great master\\s of comedy and his life The realism really comes home with the little things the fantasy of the guard which rather than use the traditional \\dream\\ techniques remains solid then disappears It plays on our knowledge and our senses particularly with the scenes concerning Orton and Halliwell and the sets particularly of their flat with Halliwell\\s murals decorating every surface are terribly well done'],
      dtype='<U13737')

In [7]:
X = []
y = []

In [8]:
for label, features in newlist:
    X.append(features)
    y.append(label)
#X = np.array(X) #3 VALUES

In [9]:
NewX = np.asarray(X).reshape(50000,1)
NewY = np.asarray(y).reshape(50000,1)
print(NewX[1])
print(NewY.shape)

['A wonderful little production The filming technique is very unassuming very oldtimeBBC fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece The actors are extremely well chosen Michael Sheen not only has got all the polari but he has all the voices down pat too You can truly see the seamless editing guided by the references to Williams\\ diary entries not only is it well worth the watching but it is a terrificly written and performed piece A masterful production about one of the great master\\s of comedy and his life The realism really comes home with the little things the fantasy of the guard which rather than use the traditional \\dream\\ techniques remains solid then disappears It plays on our knowledge and our senses particularly with the scenes concerning Orton and Halliwell and the sets particularly of their flat with Halliwell\\s murals decorating every surface are terribly well done']
(50000, 1)


In [10]:
from gensim.models import Word2Vec

In [11]:
# train model
model = Word2Vec(NewX, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
print(model)
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Word2Vec(vocab=49582, size=100, alpha=0.025)


In [12]:
model.wv.vectors.shape

(49582, 100)

In [ ]:
model.wv.vectors[0]

In [15]:
X = model.wv.vectors

In [23]:
def data_genertor(data, window , distance=0):
    x=[]
    y=[]
    for i in range(len(data)-window-distance):
        x.append(data[i:i+window])
        y.append(data[i+window+distance])
    return np.array(x),np.array(y)

In [26]:
train_x,train_y=data_genertor(model.wv.vectors[0:45000], 10 , 0)
#test_x,test_y=data_genertor(model.wv.vectors[0:45000], 10 , 0)

In [40]:
print(train_x.shape)
print(train_y.shape)

(49572, 10, 100)
(49572, 100)


In [27]:
RNN=tf.keras.Sequential()
RNN.add(tf.keras.layers.LSTM(units=1,input_shape=(49582,100),activation="relu"))
RNN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1)                 408       
Total params: 408
Trainable params: 408
Non-trainable params: 0
_________________________________________________________________


In [28]:
RNN.compile(optimizer='adam', loss='mean_squared_error')

In [29]:
#  input shape must be [samples, time steps, features]
#train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1],1))
#test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1],1))

RNN.fit(train_x,train_y,
                epochs=100,
                batch_size=100,
                shuffle=False
                )

Epoch 1/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3466e-06
Epoch 2/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3309e-06
Epoch 3/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3308e-06
Epoch 4/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3307e-06
Epoch 5/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3307e-06
Epoch 6/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3307e-06
Epoch 7/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3307e-06
Epoch 8/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3307e-06
Epoch 9/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3307e-06
Epoch 10/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3307e-06
Epoch 11/100
450/450 [==============================] - 2s 4ms/step - loss: 8.3307e-06
Epoch 12/100
450/450 [==============================

In [30]:
test_x,test_y=data_genertor(model.wv.vectors[45000:49572], 10 , 0)

In [31]:
Salidas=RNN.predict(test_x,verbose=True)

143/143 [==============================] - 0s 1ms/step


# Con Keras

In [6]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)


In [7]:
y_train = keras.utils.to_categorical( y_train , num_classes=2 ) 
y_test = keras.utils.to_categorical( y_test , num_classes=2 ) 

In [9]:
X_train = pad_sequences( X_train , maxlen=120 , padding='pre' ) 
X_test = pad_sequences( X_test , maxlen=120 , padding='pre' )

In [11]:
dropout_rate = 0.3
batch_size = 1000
activation_func = keras.activations.relu

SCHEMA = [

    Embedding( 5000 , 10, input_length=120 ),
    LSTM( 32 ) ,
    Dropout(dropout_rate),
    Dense( 32 , activation=activation_func ) ,
    Dropout(dropout_rate),
    Dense( 2 , activation=keras.activations.softmax )
    
]

model = keras.Sequential(SCHEMA)
model.compile(
    optimizer=keras.optimizers.Adam() ,
    loss=keras.losses.categorical_crossentropy ,
    metrics=[ 'accuracy' ]
)

In [12]:
model.fit(X_train , y_train , batch_size=batch_size , epochs=10 )

Epoch 1/10
25/25 [==============================] - 6s 242ms/step - loss: 0.6918 - accuracy: 0.5341
Epoch 2/10
25/25 [==============================] - 6s 238ms/step - loss: 0.6041 - accuracy: 0.7197
Epoch 3/10
25/25 [==============================] - 6s 239ms/step - loss: 0.4141 - accuracy: 0.8267
Epoch 4/10
25/25 [==============================] - 6s 237ms/step - loss: 0.3112 - accuracy: 0.8760
Epoch 5/10
25/25 [==============================] - 6s 238ms/step - loss: 0.2675 - accuracy: 0.8970
Epoch 6/10
25/25 [==============================] - 6s 238ms/step - loss: 0.2443 - accuracy: 0.9093
Epoch 7/10
25/25 [==============================] - 6s 239ms/step - loss: 0.2290 - accuracy: 0.9180
Epoch 8/10
25/25 [==============================] - 6s 237ms/step - loss: 0.2207 - accuracy: 0.9224
Epoch 9/10
25/25 [==============================] - 6s 240ms/step - loss: 0.2094 - accuracy: 0.9290
Epoch 10/10
25/25 [==============================] - 6s 237ms/step - loss: 0.2012 - accuracy: 0.9328

In [13]:
model.evaluate(X_test , y_test )

782/782 [==============================] - 7s 9ms/step - loss: 0.4110 - accuracy: 0.8445


[0.4109703004360199, 0.8445199728012085]